# 3221 NonLinear buckling of a simple I girder model in shell elements
<i>Creates a 3D Shell model of a plate I Girder with varying sections and perfoms a geometric and material nonlinear buckling analysis</i>
***

Define a list of segment lengths so the girder can have changes in sections

In [ ]:
''' Inputs '''
segments_lengths = [6000, 5000, 6000]
mesh_size = 500
imperfection = sum(segments_lengths) / 1000 # Magnitude of imperfection
steel_yield = 250 #MPa
variable_load_udl = 50 #N/mm
solve_analyses = True

Create a girder class to hold the section dimensions for each segment.</br>
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [ ]:
class GirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Create a list of section definitions, we'll need one per segment + 1 for the end.

In [ ]:
sections :list[GirderSection] = []
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections)-1)

Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [ ]:
stiffener_thicknesses = [20, 10, 10, 20]

# Ensure number of stiffener thicknesses matches the number of segments
assert(len(sections) == len(stiffener_thicknesses))

Connect to LUSAS Modeller

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

if lusas.existsDatabase():
   raise Exception("This script will create a new model. Please save and close the current model and try again")

lusas.newProject("Structural", "Nonlinear Buckling.mdl")

db = lusas.database() # Get a reference to the current model database for convenience - note a blank model needs to be open
# Set the vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Units N,mm
db.setModelUnits("N,mm,t,s,C")


Create a helper function to create a point in the model. The db.createPoint() function returns an IFObjectSet which can contain multiple objects. Since we have only created a single point we'll get a reference to the point and return it

In [ ]:
def create_point_in_modeller(x:float, y:float, z:float) -> 'IFPoint':
    return win32.CastTo(db.createPoint(x, y, z).getObject("Point"), "IFPoint")

Create all the points for each section, saving the returned point in the Girder section definition

In [ ]:
s:GirderSection
x = 0
for i in range(0, len(sections)):
    # Determine the longitudinal position (x coord) for each section
    if i > 0 : x += segments_lengths[i-1]
    # Determine the section to use at each longitudinal position
    s = sections[i]
    # Create the bottom flange points
    s.bf_points.append(create_point_in_modeller(x, +s.bottom_flange_breadth/2, 0.0))
    s.bf_points.append(create_point_in_modeller(x, 0.0,                        0.0))
    s.bf_points.append(create_point_in_modeller(x, -s.bottom_flange_breadth/2, 0.0))
    # Create the top flange points
    s.tf_points.append(create_point_in_modeller(x, +s.top_flange_breadth/2, s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, 0.0,                     s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, -s.top_flange_breadth/2, s.web_depth))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [ ]:
def create_surface_in_lusas(pnts: list, thk:float, ecc:float, group:str):

    geometry_data = lusas.geometryData()
    geometry_data.setAllDefaults()
    geometry_data.setCreateMethod("coons")
    geometry_data.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet()
    for p in pnts:
        obs.add(p)

    objs = obs.createSurface(geometry_data).getObjects("Surface")
    grp = db.getGroupByName(group)
    grp.add(objs, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(objs)


Create groups to contain the girder surfaces

In [ ]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")

Here we'll create the surfaces between each section definition using the points we created earlier

In [ ]:
for i in range(0, len(segments_lengths)):

    s1:GirderSection = sections[i]   # Section definition at the start of the segment
    s2:GirderSection = sections[i+1] # Section definition at the end of the segment

    create_surface_in_lusas([ s1.bf_points[0], s2.bf_points[0], s2.bf_points[1], s1.bf_points[1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[2], s1.bf_points[2] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s1.tf_points[1], s2.tf_points[1], s2.bf_points[1] ], s1.web_thk,           0.0, "Web")
    create_surface_in_lusas([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[0], s1.tf_points[0] ], s1.top_flange_thk,    0.0, "Top Flange")
    create_surface_in_lusas([ s1.tf_points[2], s2.tf_points[2], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")

<div class="alert alert-block alert-info">
<b>Note:</b> The order in which the points are specified to define the surface dictates the direction of the surface z axis. The surface z axis defines the top and bottom faces of shell elements. Here the top and bottom flange surfaces are defined in different orders such that when we plot the "top" stresses we will see the extreme fibre stresses for the top and bottom flanges. It is important that each flange has a consistent z orientation to avoid confusing result plots.
</div>

In [ ]:
# Display the surfaces axes.
lusas.view().geometry().showSurfaceAxes(True)

Create bearing stiffeners at the end sections

In [ ]:
for i, thk in enumerate(stiffener_thicknesses):

    if thk > 0:

        s:GirderSection = sections[i]   # Section definition at the start of the segment
        create_surface_in_lusas([ s.bf_points[0], s.tf_points[0], s.tf_points[1], s.bf_points[1] ], thk, 0.0, "Web Stiffeners")
        create_surface_in_lusas([ s.bf_points[1], s.tf_points[1], s.tf_points[2], s.bf_points[2] ], thk, 0.0, "Web Stiffeners")


We'll need to create and assign a shell mesh attribute

In [ ]:
# Create the mesh attribute
mesh_attr = db.createMeshSurface("SMsh1")
mesh_attr.setRegularSize("QTS4", mesh_size, True)

# Assign to all 
mesh_attr.assignToAll(lusas.assignment().setAllDefaults())

# Mesh the model
db.resetMesh()
db.updateMesh()

And a linear material attribute

In [ ]:
# Create and assign the steel material attribute, we'll do this all inline so we dont hold onto any references
db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6).assignToAll(lusas.assignment().setAllDefaults())

And a nonlinear material, this will be assigned later in the nonlinear analysis

In [ ]:
# Create and assign the steel material attribute, we'll do this all inline so we dont hold onto any references
steel_nl = db.createIsotropicMaterial("Steel NL", 200_000, 0.3, 7.8e-9, 10e-6)
steel_nl.addPlasticStressPotentialVonMises(steel_yield, 0.0, 0.0)

Support attributes

In [ ]:
pinned = db.createSupportStructural("Fixed in XYZ").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in YZ").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")

# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have been created for each section

# Do this for the start and end sections
for i, section in enumerate([sections[0], sections[-1]]):

    p0 = section.bf_points[0] # point at the +ve flange edge
    p1 = section.bf_points[1] # point at the centre of the web.
    p2 = section.bf_points[2] # point at the -ve flange edge

    # Loop through connected lines 
    for line in p1.getHOFs():
        line = win32.CastTo(line, "IFLine")
        # Points of the connected line
        ps1 = line.getStartPoint()
        ps2 = line.getEndPoint()
        # Check if the connected line points are those defining the bottom flange
        # If so then we can assign the relevant support attribute
        if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
            pinned.assignTo(line) if i == 0 else slide.assignTo(line)


Set the view to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Load attribute used to model live load - this will be applied along the top flange
load_attr = db.createLoadingGlobalDistributed("Global Distributed Load")
load_attr.setGlobalDistributed("Length", 0.0, 0.0, -variable_load_udl, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)

Create an eigenvalue buckling analysis to provide a deformed shape that will be used to model imperfection in the nonlinear analysis

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Eigenvalue Buckling Analysis", False)
# Create a loadcase in the analysis
first_loadcase = db.createLoadcase("Eig Buckling", analysis.getName())
first_loadcase.addGravity(True)
# Set the loadcase controls to be a Buckling analysis searching for 5 buckling modes between 0 and 20. 
first_loadcase.setEigenvalueRangeControl("Buckling", "Buckling load", 20, 0.0, 5)
eigen_control = first_loadcase.getEigenvalueControl()
eigen_control.setValue("MaxIterations", 0)
eigen_control.setValue("NormalisationProcedure", "Unity")
eigen_control.setValue("Eigensolver", "Default")


<div class="alert alert-block alert-info">
<b>Note:</b> The eigenvalue range control is used to avoid any potential negative modes. If no modes are found you must increase the range
</div>

Create the nonlinear analysis with two loadcases, permanent load applied in the first loadcasse with variale load applied until the girder fails. This way we can determine a variable load capacity

In [ ]:
# Create a new analysis without an initial loadcase
analysis = db.createAnalysisStructural("Nonlinear Buckling Analysis", False)
# Set the analysis to start with the deformed mesh from the first mode of the eigen analysis
analysis.setDeformedMeshStart(first_loadcase, 1, imperfection)

# Create a loadcase in the analysis
first_nl_loadcase = db.createLoadcase("Permanent Load", analysis.getName())
first_nl_loadcase.addGravity(True)

# Set the nonlinear control on the first loadcase to apply the load in a manual (single step)
transient_control = first_nl_loadcase.setTransientControl(0).getTransientControl().setNonlinearManual().setOutput().setConstants()


In [ ]:
second_nl_loadcase = db.createLoadcase("Variable Load", analysis.getName())

transient_control = second_nl_loadcase.setTransientControl(0).getTransientControl().setOutput().setConstants()
transient_control.setNonlinearAutomatic(0.1)             # Start load factor of 0.1
transient_control.setValue("Iterations", 0)              # Don't change the load factor
transient_control.setValue("MaxChangeInLoadFactor", 0.1) # Constant load increment of 0.1
transient_control.setValue("MaxLoadFactor", 2.5)         # Maximum load factor of 2.5

# Set the option to use Geometric Nonlinearity in the Nonlinear analysis
db.options().setBoolean("Option 87", True, False, analysis.getName())

Assign the nonlinear material only in the nonlinear analysis

In [ ]:
steel_nl.assignTo("Surface", lusas.assignment().setAllDefaults().setLoadset(first_nl_loadcase))

Assign the variable load to the top of the girder

In [ ]:
# Type definition
gs:GirderSection 

# List of the lines in the top flange
centre_lines = []

# Select the 
for i in range(0, len(sections)):
    gs = sections[i]
    point = gs.tf_points[1]

    for hof in point.getHOFs():
        if hof.getTypeCode() == 2 : # Line in the top flange
            
            line = win32.CastTo(hof, "IFLine")
            
            # Check it is aligned with and paralle to the x axis
            if abs(line.getEndPosition()[1] - line.getStartPosition()[1]) < 1e-3 and \
               abs(line.getEndPosition()[2] - line.getStartPosition()[2]) < 1e-3:

                centre_lines.append(line)


load_attr.assignTo(centre_lines, lusas.assignment().setAllDefaults().setLoadset(first_loadcase))
load_attr.assignTo(centre_lines, lusas.assignment().setAllDefaults().setLoadset(second_nl_loadcase))


Solve all the analyses

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
# This will serve as a basic model check, that reactions, deflections, stresses under self weight are all correct.
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)

In [ ]:
if solve_analyses:
    db.getAnalysis("Analysis 1").solve(True) # Useful model check
    db.getAnalysis("Eigenvalue Buckling Analysis").solve(True)
    db.getAnalysis("Nonlinear Buckling Analysis").solve(True)
    db.openAllResults(False)

    # Set active the first mode of buckling
    loadset = db.getLoadset("Mode 1", 2)
    lusas.view().setActiveLoadset(loadset)